In [1]:
#Final Project in ACM
#Students: Hoti Erza
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

class Graph:

    def __init__(self, initial_nodes, m, special_nodes_number, initial_fitness, all_nodes, file_name):
        
        self.initial_nodes = initial_nodes  # initial nodes
        self.m = m  # edges for new nodes
        self.special_nodes_number = special_nodes_number # number of special node ( special <= intial_nodes)
        self.initial_fitness = initial_fitness
        self.all_nodes = all_nodes #all number of nodes that will has graph
        self.file_name = file_name
        self.graph = {}  # adj. list

        self.node_pref_attach = []  # pref attachment list

        # init of ADJ LIST
        for i in range(self.initial_nodes):
            self.insert_node()
        
        self.connect_graph() #we cannect nodes of graph

        #list of normal node's degree
        self.normal_degree_list = []
        
        
        self.times_step = [] #time step list
        self.special_fitness = [] #list of special fitness 
        
        self.specials_degree = [] #list of all specials degree
        self.specials_fitness = []
        
        self.all_degree = [] #all nodes degree
        
        for i in range(self.initial_nodes):
            self.all_degree.append(self.degree(i))
        
        for x in range(self.special_nodes_number):
            self.special_fitness.append(initial_fitness/self.degree(x))
            self.specials_degree.append(list())
            self.specials_fitness.append(list())
    
        self.grown_of_graph() #function that will grow the graph
        
        self.plot_grown_of_graph()  #function that will plot degrees  
        
               
    def grown_of_graph(self):
        t = 0 #step
        for x in range(self.initial_nodes, self.all_nodes):
            
            self.set_pref_attachment() #find preferential attachment for each node
        
            neighbors = self.get_neighbors_of_new_node(self.node_pref_attach, m) #get neighbors for new node
            self.graph[x] = set() #set new node in graph

            #connect new node with negihbors
            for node in list(neighbors):
                self.graph[x].add(node)
                self.graph[node].add(x)
                self.all_degree[node] = self.degree(node)
                if node < self.special_nodes_number:
                    self.set_special_fitness(node, self.degree(node))
                
            for special in range(self.special_nodes_number):                  
                self.specials_degree[special].append(self.degree(special))
                self.specials_fitness[special].append(self.special_fitness[special])
                
            t += 1
            self.times_step.append(t)
            self.all_degree.append(self.degree(x))
            self.normal_degree_list.append(self.degree(self.initial_nodes)) #the degree of node 10 as normal node
            
    def set_special_fitness(self, node, degree):
        special_fitness = ((self.special_fitness[node])/degree) #fitness depends on degree of node
        self.special_fitness[node] = special_fitness

    #preferential attachment
    def set_pref_attachment(self):
        self.node_pref_attach = []
        
        #list of 0.2 fitness for normal nodes
        normal_nodes_fitness = [initial_fitness] * (self.get_num_nodes()-self.special_nodes_number) 
        fitness_list = self.special_fitness + normal_nodes_fitness #all nodes's fitness
    
        sum_of_degree_fitness = sum([x*y for x,y in zip(fitness_list,self.all_degree)])
        self.node_pref_attach = [(x*y)/sum_of_degree_fitness for x,y in zip(self.all_degree, fitness_list)]
            

    #get neighbors of new node 
    def get_neighbors_of_new_node(self, pref_attachment, neighbors_size):
      
        if neighbors_size > len(pref_attachment):
            raise ValueError("The number of neighbor should not be greater than the graph size")
        else:
            neighbors = np.random.choice(list(self.graph.keys()),
                                                  neighbors_size, 
                                                  replace=False,
                                                  p=list(pref_attachment))
            return neighbors
        

    #connect nodes of graph as a ring
    def connect_graph(self):
        nodes = list(self.graph.keys())
        for x in nodes:
            if x != len(nodes)-1:
                self.add_edge(x,x+1)
            else:
                self.add_edge(x,0)
                
    def get_num_nodes(self):
        return len(self.graph) #returns the number of nodes

    def insert_node(self):
        self.graph[len(self.graph)] = set()  #add new element in adj list

    def degree(self,u):
        return len(self.graph[u])  #returns degree of node u

    def add_edge(self,u,v):
        self.graph[u].add(v)
        self.graph[v].add(u)
    
    def plot_function(self, x, y, title, type, legend, legend_loc): 
        
        for i in range(len(y)):
            plt.plot(x,y[i])

        plt.xscale(type)
        plt.yscale(type)
        plt.title(title)
        plt.legend([legend], loc = legend_loc)
        
    def plot_function_without_legend(self, x, y, title, type):

        for i in range(len(y)):
            plt.plot(x,y[i])

        plt.xscale(type)
        plt.yscale(type)
        plt.title(title)
    
    def plot_grown_of_graph(self):
                
        fig = plt.figure(figsize=(10,10))
        plt.subplot(2, 2, 1)
        plt.plot(self.times_step,self.normal_degree_list)
        self.plot_function(self.times_step, self.specials_degree, "Degree of Nodes (Linear)", "linear", "n", "upper left")
        
        plt.subplot(2, 2, 2)
        plt.plot(self.times_step,self.normal_degree_list)
        self.plot_function(self.times_step, self.specials_degree, "Degree of Nodes (Log)", "log", "n", "upper left")
    
        
        plt.subplot(2, 2, 3)
        
        self.plot_function_without_legend(self.times_step, self.specials_fitness, "Fitness of Special Nodes (Linear)", "linear")


        plt.subplot(2, 2, 4)
        self.plot_function_without_legend(self.times_step, self.specials_fitness, "Fitness of Special Nodes (Log)", "log")
        
        plt.savefig(self.file_name)



initial_nodes = 10
m = 4
special = 3
initial_fitness = 0.2
all_nodes = 15000


g = Graph(initial_nodes,m,special,initial_fitness, all_nodes, 'plot_result.png')